# Preparation stuff

## Connect to Drive

In [1]:
connect_to_drive = False

In [2]:
#Run command and authorize by popup --> other window
if connect_to_drive:
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)

## Install packages

In [3]:
if connect_to_drive:
    #Install FS code
    !pip install  --upgrade --force-reinstall git+https://github.com/federicosiciliano/easy_lightning.git

    !pip install pytorch_lightning

## IMPORTS

In [4]:
#Put all imports here
import numpy as np
from copy import deepcopy
import os
import sys

## Define paths

In [5]:
#every path should start from the project folder:
project_folder = "../"
if connect_to_drive:
    project_folder = "/content/gdrive/Shareddrives/<SharedDriveName>" #Name of SharedDrive folder
    #project_folder = "/content/gdrive/MyDrive/<MyDriveName>" #Name of MyDrive folder

#Config folder should contain hyperparameters configurations
cfg_folder = os.path.join(project_folder,"cfg")

#Data folder should contain raw and preprocessed data
data_folder = os.path.join(project_folder,"data")
raw_data_folder = os.path.join(data_folder,"raw")
processed_data_folder = os.path.join(data_folder,"processed")

#Source folder should contain all the (essential) source code
source_folder = os.path.join(project_folder,"src")

#The out folder should contain all outputs: models, results, plots, etc.
out_folder = os.path.join(project_folder,"out")
img_folder = os.path.join(out_folder,"img")

## Import own code

In [6]:
#To import from src:

#attach the source folder to the start of sys.path
sys.path.insert(0, project_folder)

#import from src directory
# from src import ??? as additional_module
from easy_lightning import easy_rec as additional_module #REMOVE THIS LINE IF IMPORTING OWN ADDITIONAL MODULE

from easy_lightning import easy_exp, easy_rec, easy_torch #easy_data

In [7]:
from datasets import load_dataset

if not os.path.exists(os.path.join(data_folder, "raw", "tim")):
    # download the dataset
    dataset = load_dataset("shba93/tim-rec", split="train")
    # save it to the raw data folder as csv
    os.makedirs(os.path.join(data_folder, "raw", "tim"))
    dataset.to_csv(os.path.join(raw_data_folder, "tim", "dataset.csv"), index=False)
    #dataset = load_dataset("shba93/tim-rec")["train"].to_pandas()

# MAIN

## Train

### Data

In [8]:
cfg = easy_exp.cfg.load_configuration("config_rec")

In [9]:
exp_found, experiment_id = easy_exp.exp.get_set_experiment_id(cfg)

In [10]:
data, maps = easy_rec.preparation.prepare_rec_data(cfg)

Ratings data already exists. Skip pre-processing
Densifying index


/home/federico/Scrivania/RecSys TIM/venv/lib/python3.10/site-packages/easy_lightning/easy_rec/data_generation_utils.py:459: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


Splitting: leave_n_out


/home/federico/Scrivania/RecSys TIM/venv/lib/python3.10/site-packages/easy_lightning/easy_rec/data_generation_utils.py:462: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [ ]:
data["train_rating"] = []

array([list([]), list([]), list([]), ..., list([]), list([]), list([0])],
      shape=(4074,), dtype=object)

In [11]:
# for split_name in ["train","val","test"]:
#     data[f"{split_name}_uid"] = deepcopy(data["uid"])
#     to_keep = []
#     for i in range(len(data[f"{split_name}_sid"])):
#         if len(data[f"{split_name}_sid"][i])>0:
#             to_keep.append(i)
#     for key in ["uid","sid","rating","timestamp"]:
#         data[f"{split_name}_{key}"] = [data[f"{split_name}_{key}"][i] for i in to_keep]
# data.pop("uid")

In [12]:
# for split_name in ["train","val","test"]:
#     data["train_relevance"]

In [13]:
loaders = easy_rec.preparation.prepare_rec_dataloaders(cfg, data, maps)

In [14]:
main_module = easy_rec.preparation.prepare_rec_model(cfg, maps)

Seed set to 42
/home/federico/Scrivania/RecSys TIM/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


In [15]:
trainer = easy_torch.preparation.complete_prepare_trainer(cfg, experiment_id, additional_module=easy_rec)

Seed set to 42
Seed set to 42
Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [16]:
model = easy_torch.preparation.complete_prepare_model(cfg, main_module, easy_rec)

Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42


In [17]:
experiment_id

'lC0JBI3pAYbjnPEM'

In [18]:
easy_torch.process.test_model(trainer, model, loaders, test_key=["val","test","train"])

Seed set to 42
You are using a CUDA device ('NVIDIA RTX A4000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
/home/federico/Scrivania/RecSys TIM/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:476: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0             DataLoader 1             DataLoader 2
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_NDCG_@10         0.9203163385391235       0.8574556112289429               nan
      test_NDCG_@20         0.9373889565467834       0.9052756428718567               nan
      test_NDCG_@5          0.8929141163825989       0.7851969599723816               nan
   test_Precision_@10       0.9578061103820801        0.936939001083374      0.0064459932036697865
   test_Precision_@20       0.9639658331871033       0.9658048152923584      0.0032229966018348932
    test_Precision_@5       0.9414355754852295       0.8778535723686218      0.012891986407339573
     test_Recall_@10        0.09567049145698547      0.0550667829811573               nan
     tes

In [19]:
# Train the model using the prepared trainer, model, and data loaders
easy_torch.process.train_model(trainer, model, loaders, val_key=["val","test"])

# Early stopping with Tune schedulers may not run anything after training

Seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name        | Type                        | Params | Mode 
--------------------------------------------------------------------
0 | main_module | SASRec                      | 63.9 K | train
1 | loss        | SequentialBCEWithLogitsLoss | 0      | train
2 | metrics     | RobustModuleDict            | 0      | train
--------------------------------------------------------------------
63.9 K    Trainable params
0         Non-trainable params
63.9 K    Total params
0.256     Total estimated model params size (MB)
101       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/federico/Scrivania/RecSys TIM/venv/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (31) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [20]:
easy_torch.process.test_model(trainer, model, loaders, test_key=["val","test","train"])

Seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0             DataLoader 1             DataLoader 2
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_NDCG_@10         0.9817453622817993       0.9776284098625183               nan
      test_NDCG_@20         0.9869676232337952       0.9846146106719971               nan
      test_NDCG_@5          0.9740223288536072       0.9685872197151184               nan
   test_Precision_@10       0.9998815655708313       0.9996705651283264      0.0019163762917742133
   test_Precision_@20       0.9991469979286194       0.9997317790985107      0.0009581881458871067
    test_Precision_@5       0.9998578429222107       0.9996829628944397      0.0038327525835484266
     test_Recall_@10        0.0998750552535057       0.05875510349869728              nan
     te

In [21]:
#print("Experiment already found:", exp_found, "----> The experiment id is:", experiment_id)

# if exp_found and if_exp_found == "skip":
#     #print("Skipping experiment")
#     return

# Save experiment (done here cause Early stopping with Tune schedulers may not run anything after training)
easy_exp.exp.save_experiment(cfg)